# Libraries

In [1]:
!pip install transformers
from IPython.display import clear_output
clear_output()

In [2]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import math
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

# Load Data

In [3]:
df = pd.read_csv('/kaggle/input/undang-undan-v3/final-dataset.csv')

In [4]:
df.head()

,NAMA_DOKUMEN,NO_PASAL,NO_AYAT,BUNYI_AYAT,NAMA_DOKUMEN_LAIN,NO_PASAL_LAIN,NO_AYAT_LAIN,BUNYI_AYAT_LAIN,TOPIC
0,Undang-Undang_No.40_Tahun_2014,3,2,Perusahaan asuransi jiwa syariah hanya dapat m...,Undang-Undang_No.40_Tahun_2014,53,1,Perusahaan Asuransi dan Perusahaan Asuransi Sy...,35
1,Peraturan_Pemerintah_No.25_Tahun_2020,38,3,Ketentuan lebih lanjut mengenai persyaratan un...,Peraturan_Pemerintah_No.25_Tahun_2020,64,1,Dana Tapera yang bersumber dari dana Fasilitas...,2
2,PERMEN_DAGRI_52_2012,20,4,Penyertaan modal dan pemberian pinjaman sebaga...,Peraturan_Menteri_Keuangan_nomor_218_PMK.06_20...,23,7,Penilaian atas aspek operasional penerima inve...,24
3,Undang-Undang_No.9_Tahun_2018,19,2,Mitra Instansi Pengelola PNBP sebagaimana dima...,Undang-Undang_No.9_Tahun_2018,47,1,Terhadap Wajib Bayar yang menghitung sendiri k...,25
4,Undang-Undang_No.21_Tahun_2008,35,4,Bank Indonesia dapat menetapkan pengecualian t...,Undang-Undang_No.21_Tahun_2008,52,1,Bank Syariah dan UUS wajib menyampaikan segala...,13


In [5]:
ayat_df = df[['BUNYI_AYAT','BUNYI_AYAT_LAIN']]
ayat_df.head()

,BUNYI_AYAT,BUNYI_AYAT_LAIN
0,Perusahaan asuransi jiwa syariah hanya dapat m...,Perusahaan Asuransi dan Perusahaan Asuransi Sy...
1,Ketentuan lebih lanjut mengenai persyaratan un...,Dana Tapera yang bersumber dari dana Fasilitas...
2,Penyertaan modal dan pemberian pinjaman sebaga...,Penilaian atas aspek operasional penerima inve...
3,Mitra Instansi Pengelola PNBP sebagaimana dima...,Terhadap Wajib Bayar yang menghitung sendiri k...
4,Bank Indonesia dapat menetapkan pengecualian t...,Bank Syariah dan UUS wajib menyampaikan segala...


# Zero-shot Classification

In [6]:
# Set label for model
candidate_labels = ["Contradiction", "Not contradiction"]

In [7]:
# Define Model
pretrained_name = "StevenLimcorn/indo-roberta-indonli"
classifier = pipeline(
    "zero-shot-classification",
    model=pretrained_name,
    tokenizer=pretrained_name,
    device = 0
)

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [8]:
# Testing
text_1 = 'Saya suka mangga'
text_2 = 'Saya tidak suka mangga'
model_dict = classifier(f"{text_1}</s></s>{text_2}",candidate_labels,multi_label=True)
result_dict = dict(zip(model_dict.get('labels'), model_dict.get('scores')))
print(result_dict)
print(model_dict)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'Contradiction': 0.01936390995979309, 'Not contradiction': 0.009094178676605225}
{'sequence': 'Saya suka mangga</s></s>Saya tidak suka mangga', 'labels': ['Contradiction', 'Not contradiction'], 'scores': [0.01936390995979309, 0.009094178676605225]}


In [9]:
result_label = []
for i in tqdm(range(ayat_df.shape[0])):
    text_1 = ayat_df.loc[i,'BUNYI_AYAT']
    text_2 = ayat_df.loc[i,'BUNYI_AYAT_LAIN']
    input_text = f"{text_1}</s></s>{text_2}"
    try:
        model_dict = classifier(input_text,candidate_labels,multi_label=True,max_length=512,truncation=True)
        result_dict = dict(zip(model_dict.get('labels'), model_dict.get('scores')))
        result_label.append(result_dict)
    except RuntimeError:
        result_label.append({'Contradiction': 0,'Not contradiction':0})

  6%|▋         | 11529/178845 [04:52<1:11:49, 38.83it/s]/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [388,0,0], thread: [32,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [388,0,0], thread: [33,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [388,0,0], thread: [34,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [388,0,0], thread: [35,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [388,0,0], thread: [36,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectL

In [10]:
# Make dataframe from model output
label_df = pd.DataFrame(result_label)
label_df.head()

,Contradiction,Not contradiction
0,0.883111,0.832480
1,0.573134,0.510589
2,0.804407,0.704269
3,0.551624,0.538393
4,0.716556,0.663056


In [11]:
# Save to csv
label_df.to_csv('label.csv', index=False)

# Prediction with pretrained model 

In [12]:
# define model
pipe = pipeline("text-classification", model="StevenLimcorn/indo-roberta-indonli", tokenizer='StevenLimcorn/indo-roberta-indonli',device=0,max_length=512, truncation=True)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # define model                                                                               │
│ ❱ 2 pipe = pipeline("text-classification", model="StevenLimcorn/indo-roberta-indonli", token     │
│   3                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/pipelines/__init__.py:988 in pipeline       │
│                                                                                                  │
│   985 │   if device is not None:                                                                 │
│   986 │   │   kwargs["device"] = device                                                          │
│   987 │                                                                                          │
│ ❱ 988 │   return pipeline_class(model=model, framework=framework, task=task, **kwargs)           │
│   989                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:83 in      │
│ __init__                                                                                         │
│                                                                                                  │
│    80 │   function_to_apply = ClassificationFunction.NONE                                        │
│    81 │                                                                                          │
│    82 │   def __init__(self, **kwargs):                                                          │
│ ❱  83 │   │   super().__init__(**kwargs)                                                         │
│    84 │   │                                                                                      │
│    85 │   │   self.check_model_type(                                                             │
│    86 │   │   │   TF_MODEL_FOR_SEQUENCE_CLASSIFICATION_MAPPING                                   │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:780 in __init__           │
│                                                                                                  │
│    777 │   │   self.framework = framework                                                        │
│    778 │   │                                                                                     │
│    779 │   │   if self.framework == "pt" and device is not None and not (isinstance(device, int  │
│ ❱  780 │   │   │   self.model.to(device)                                                         │
│    781 │   │                                                                                     │
│    782 │   │   if device is None:                                                                │
│    783 │   │   │   # `accelerate` device map                                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:1902 in to                │
│                                                                                                  │
│   1899 │   │   │   │   " model has already been set to the correct devices and casted to the co  │
│   1900 │   │   │   )                                       

In [ ]:
# test the model
text_1 = ayat_df.loc[108]['BUNYI_AYAT']
text_2 = ayat_df.loc[108]['BUNYI_AYAT_LAIN']

pipe(f"{text_1}</s></s>{text_2}")

In [ ]:
# Labelling with pre-trained model class
result_label = []
for i in tqdm(range(ayat_df.shape[0])):
    text_1 = ayat_df.loc[i,'BUNYI_AYAT']
    text_2 = ayat_df.loc[i,'BUNYI_AYAT_LAIN']
    input_text = f"{text_1}</s></s>{text_2}"
    pred = pipe(input_text)[0]
    result_label.append(pred)

In [ ]:
# Make dataframe from model prediction
label_2_df = pd.DataFrame(result_label)
label_2_df.head()

In [ ]:
# Convert to csv
label_2_df.to_csv('label-2.csv', index=False)